<a href="https://colab.research.google.com/github/HSE-LAMBDA/mldm-2019/blob/master/day-3/BiasVariance_ModelAssessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bias-Variance decomposition

In [0]:
import numpy as np
import matplotlib.pyplot as plt

In [0]:
def true_dep(x):
  return np.cos((x - 0.2)**2) + 0.2 / (1 + 50 * (x - 0.3)**2)

x_true = np.linspace(0, 1, 100)
y_true = true_dep(x_true)

def generate_n_datasets(num_datasets, dataset_length, noise_power=0.02):
  shape = (num_datasets, dataset_length, 1)
  x = np.random.uniform(size=shape)
  y = true_dep(x) + np.random.normal(scale=noise_power, size=shape)
  return x, y

In [0]:
x, y = generate_n_datasets(1, 30)
plt.scatter(x.squeeze(), y.squeeze(), s=20, c='orange')
plt.plot(x_true, y_true, c='c', linewidth=1.5);

In [0]:
from copy import deepcopy
from tqdm import tqdm, trange

In [0]:
def calc_bias2_variance(model, datasets_X, datasets_y):
  preds = []
  for X, y in tqdm(zip(datasets_X, datasets_y), total=len(datasets_X)):
    m = deepcopy(model)
    m.fit(X, y)
    preds.append(m.predict(x_true[:,np.newaxis]).squeeze())
  preds = np.array(preds)
  mean_pred = preds.mean(axis=0)
  bias2 = (y_true - mean_pred)**2
  variance = ((preds - mean_pred[np.newaxis,...])**2).mean(axis=0)

  return bias2, variance, preds

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

In [0]:
powers = np.arange(1, 9)

bias2, variance, preds = [], [], []
for p in powers:
  model = Pipeline([
      ('poly', PolynomialFeatures(degree=p)),
      ('linear', LinearRegression())
  ])

  b2, v, p = calc_bias2_variance(model, *generate_n_datasets(10000, 30))
  bias2.append(b2)
  variance.append(v)
  preds.append(p)

bias2 = np.array(bias2)
variance = np.array(variance)

In [0]:
ncols = 4
nrows = int(np.ceil(len(powers) / ncols))

plt.figure(figsize=(18, 3.5 * nrows))

yrange = y_true.max() - y_true.min()

for i, (pred, pow) in tqdm(enumerate(zip(preds, powers), 1)):
  plt.subplot(nrows, ncols, i)
  for p in pred[np.random.choice(len(pred), size=200, replace=False)]:
    plt.plot(x_true, p, linewidth=0.05, c='b');
  plt.plot(x_true, y_true, linewidth=3, label='Truth', c='r')
  plt.ylim(y_true.min() - 0.5 * yrange, y_true.max() + 0.5 * yrange)
  plt.title('power = {}'.format(pow))
  plt.legend();

In [0]:
plt.plot(powers, bias2.mean(axis=1), label='bias^2')
plt.plot(powers, variance.mean(axis=1), label='variance')
plt.legend()
plt.yscale('log')
plt.xlabel('power');

In [0]:
plt.figure(figsize=(18, 15))
iplot = 1
for i in range(0, bias2.shape[1], 10):
  plt.subplot(4, 3, iplot); iplot += 1
  plt.plot(powers, bias2[:,i], label='bias^2')
  plt.plot(powers, variance[:,i], label='variance')
  plt.legend()
  plt.yscale('log')
  plt.xlabel('power');

----------

# Notes on Cross-Validation

In [0]:
N = 50
p = 1000

n_cv = 5

num_best_features = 100
n_neighbors = 1

n_experiments = 100

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

def run_experiment():
  X = np.random.normal(size=(N, p))
  y = np.random.randint(2, size=N)

  abs_cov = np.abs(((X - X.mean(axis=0)) * ((y - y.mean())[:,np.newaxis])).sum(axis=0))
  best_features = np.argsort(abs_cov)[-num_best_features:]
  X_best = X[:,best_features]
  
  kf = KFold(n_splits=n_cv)

  model = KNeighborsClassifier(n_neighbors=n_neighbors)

  cv_errors = []
  for i_train, i_test in kf.split(X_best):
    model.fit(X_best[i_train], y[i_train])
    cv_errors.append(accuracy_score(y[i_test], model.predict(X_best[i_test])))
  
  return np.mean(cv_errors)


results = [run_experiment() for _ in trange(n_experiments)]

plt.hist(results, bins=10);

In [0]:
def run_experiment_2():
  X = np.random.normal(size=(N, p))
  y = np.random.randint(2, size=N)
  
  kf = KFold(n_splits=n_cv)

  model = KNeighborsClassifier(n_neighbors=n_neighbors)

  cv_errors = []
  for i_train, i_test in kf.split(X):
    abs_cov = np.abs(((X[i_train] - X[i_train].mean(axis=0)) * 
                      ((y[i_train] - y[i_train].mean())[:,np.newaxis])).sum(axis=0))
    best_features = np.argsort(abs_cov)[-num_best_features:]
    X_best = X[:,best_features]

    model.fit(X_best[i_train], y[i_train])
    cv_errors.append(accuracy_score(y[i_test], model.predict(X_best[i_test])))
  
  return np.mean(cv_errors)


results = [run_experiment_2() for _ in trange(n_experiments)]

plt.hist(results, bins=10);

# Notes on Test error

In [0]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.base import clone

In [0]:
def gen_data():
  X, y = make_blobs(n_samples=20000, centers=[[0., 0.5], [1., 0.]])
  X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.01)
  return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = gen_data()
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
plt.scatter(*X_train.T, c=y_train);

In [0]:
def run_cv(model, X, y, error_func, n_cv=5):
  kf = KFold(n_splits=n_cv)
  errors = []
  for i_train, i_test in kf.split(X):
    m = clone(model)
    m.fit(X[i_train], y[i_train])
    errors.append(error_func(y[i_test], m.predict(X[i_test])))
  model.fit(X, y)
  return np.mean(errors)

def error(y_true, y):
  return 1. - accuracy_score(y_true, y)

def true_vs_cv_error(model):
  X_train, X_test, y_train, y_test = gen_data()
  cv_error = run_cv(model, X_train, y_train, error)
  true_error = error(y_test, model.predict(X_test))
  return true_error, cv_error

In [0]:
from sklearn.svm import SVC

In [0]:
errors = np.array([true_vs_cv_error(SVC(C=1., gamma='auto'))
                   for _ in trange(100)])

In [0]:
plt.scatter(*errors.T, s=3.);
plt.xlim(0, 0.6)
plt.ylim(0, 0.6);
plt.xlabel("True error")
plt.ylabel("CV error");